## Шаг 3. Пишем бота на основе word2vec/fasttext

In [13]:
import string
import annoy
import pickle
import numpy as np

from tqdm import tqdm_notebook
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText
import re

In [5]:
# ячейка не обязательна если у вас уже создан prepared_answers.txt выполнять не надо
# assert False

#Small preprocess of the answers

question = None
written = False

with open("prepared_answers.txt", "w", encoding = 'utf-8') as fout:
    with open("Otvety.txt", "r", encoding = 'utf-8') as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


0it [00:00, ?it/s]

In [18]:
def preprocess_txt(line):
    spls = line.lower()
    spls =  re.sub(r'[^а-яА-Я]', ' ', spls)
    spls = re.sub("\sне", "не", spls)
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [19]:
question_len = []
answer_len = []
with open("prepared_answers.txt", "r", encoding = 'utf-8') as f:
    for line in tqdm_notebook(f):
        spls = line.split("\t")
        question_len.append(len(spls[0].split()))
        answer_len.append(len(spls[1].split()))

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


0it [00:00, ?it/s]

In [20]:
print(" question")
print(" median len = {}\n mean len = {:.2f}\n 0.75 quantile = {}\n 0.85 quantile = {}\n 0.95 quantile = {}\n max = {}\n".format(np.median(question_len), np.mean(question_len), np.quantile(question_len, 0.75), np.quantile(question_len, 0.85), np.quantile(question_len, 0.95), np.max(question_len)))

 question
 median len = 12.0
 mean len = 20.45
 0.75 quantile = 21.0
 0.85 quantile = 31.0
 0.95 quantile = 63.0
 max = 862



In [21]:
print(" answer")
print(" median len = {}\n mean len = {:.2f}\n 0.75 quantile = {}\n 0.85 quantile = {}\n 0.95 quantile = {}\n max = {}\n".format(np.median(answer_len), np.mean(answer_len), np.quantile(answer_len, 0.75), np.quantile(answer_len, 0.85), np.quantile(answer_len, 0.95), np.max(answer_len)))

 answer
 median len = 12.0
 mean len = 52.78
 0.75 quantile = 34.0
 0.85 quantile = 63.0
 0.95 quantile = 205.0
 max = 34225



In [22]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

count = 0
with open("Otvety.txt", "r", encoding = 'utf-8') as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        
        if count >= 300000:
            break
        count+=1

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


0it [00:00, ?it/s]

In [23]:
sentences = [i for i in sentences if len(i) >= 2]

In [24]:
sentences

[['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести'],
 ['мужик', 'йопарить', 'собачка', '50', 'кошка'],
 ['пол', 'памытьbr', 'таг', 'тип', 'каво'],
 ['вообще', 'пообщаться'],
 ['советовать', 'сися', 'завести'],
 ['слава', 'бог', 'мужик', 'завести', 'советовать', 'сватать', 'сватать'],
 ['советовать', 'завести', 'любовник', 'муж', 'кошка', 'brприветик'],
 ['парень',
  'относиться',
  'цветной',
  'линза',
  'девушка',
  'зелёный',
  'глаз',
  'голубой'],
 ['вобщий', 'прикалывать', 'тема'],
 ['этобыть',
  'редкость',
  'забавно',
  'знать',
  'фальшивка',
  'прикольно',
  'силиконовый',
  'сиська',
  'налепить',
  'синтетический',
  'волос'],
 ['делать', 'найти', '2', 'миллион', 'рубль'],
 ['счастие',
  'свалиться',
  'хороший',
  'пойти',
  'милиция',
  'заявить',
  'находка',
  'деньга',
  'тероть',
  'самый',
  'интересный',
  'неприменный',
  'искать',
  'поверьте',
  'найти',
  'видеть',
  'подобный',
  'нарваться',
  'бабушка',
  'помочий',
  'внук',
  'покупка',
  'ква

In [25]:
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=3, workers=8)
modelFT = FastText(sentences=sentences, vector_size=300, min_count=3, window=5, workers=8)

In [26]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

with open("prepared_answers.txt", "r", encoding = 'utf-8') as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        
        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1

w2v_index.build(10)
ft_index.build(10)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


0it [00:00, ?it/s]

True

In [27]:
w2v_index

In [28]:
index_map

{0: 'вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО ВАМ здесь кого советовали завести?)) . \n',
 1: 'меня вобще прикалывает эта тема :). \n',
 2: 'Если это "счастье " действительно на вас свалилось, лучше пойти в милицию и заявить о находке. Такие деньги просто так не терют, а что самое интересное их неприменно будут искать и поверьте мне найдут, видел подобное в жизни. Можно нарваться на бабушку конечно, которая хотела помоч внуку с покупкой квартиры, а можно на бандитов, которые будут с вами разговаривать иначе чем бабушка с милицией. Выбор за вами, есть еще конечно шанс, что это подарок с выше за котрый с вас никто не спросит, тогда лучше отдать хотябы 500 на благотворительность. дабы не спугнуть удачу!. \n',
 3: 'ЭБУ — электронный блок управления двигателем автомобиля, его другое название — контроллер. Он принимает информацию от многочисленных датчиков, обрабатывает ее по особым алгоритмам и, отталкиваясь от полученных данных, отдает команды исполнительным устройствам системы.. \n'

In [29]:
modelW2V.save("w2v.model")

In [30]:
import json
with open('index_map.json', 'w') as fp:
    json.dump(index_map, fp)

In [31]:
w2v_index.save ('anoy_w2v.ann')

True

In [32]:
def get_response(question, index, model, index_map, count_answer=3):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, count_answer, )
    return [index_map[i] for i in answers]

In [33]:
TEXT = "Самый большой природный заповедник?"

In [34]:
candidate_answer = get_response(TEXT, w2v_index, modelW2V, index_map, count_answer=3)

In [35]:
candidate_answer

['может глупо, но прочитав вопрос подумал о Боге. И возможности, так, чисто по человечески, на тесной кухоньки, поболтать о том, о сем..... \n',
 'В электротехнике как металл с самой большой электропроводностью применяется золото. И конечно не во всякой, преимущественно в военно-промышленном комплексе. \n',
 'Самый большой колокол в мире <br> <br>История колоколов насчитывает, как считается, более 40 веков, то есть первые колокольчики, найденные археологами, относятся еще к эпохе неолита. Бронзовые колокола, конечно, начали отливать позже: самый первый из них обнаружен в 1950 году, в Китае, в провинции Хэннань. <br> <br>Его высота всего 4,5 сантиметра. Отсюда, по шелковому пути колокольчики проникли во многие страны Европы. Самым старым, но все еще действующим колоколом является колокол в храме Киото, Япония. Он отлит в 698 году, его высота - 129 сантиметров. <br> <br>Самым большим колоколом в мире по сей день остается Царь-колокол. Правда, он не является действующим, потому что во вре

In [38]:
COUNT_ANSW = 3

In [59]:
def start(update: Update, context: CallbackContext):
    update.message.reply_text('!')

def text_msg(update: Update, context: CallbackContext):
    txt_q = update.message.text # текст который пришёл
    # тут проверяют к какому интенту принадлежит запрос, для этого надо натренировать классификатор
    # в зависимости от интента выбирают метод как отвечать либо шаблоны
    # либо с помощью поиска по базе, либо генеративная модель
    
    candidate_answer = get_response(txt_q, w2v_index, modelW2V, index_map, count_answer=1)
    # при формировании списка кандидатов ответов, можно задавать несколько, выводить с помощью
    # random.randomint сэмплируя один ответ из нескольких
    update.message.reply_text(candidate_answer[0])

In [60]:
updater = Updater("5037721599:AAFxPl5Xk0dHR_u2scLj8rA82xR8g1t38qY", use_context=True)
dispatcher = updater.dispatcher

# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", start))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, text_msg))

# Start the Bot
updater.start_polling()
updater.idle()